In [1]:
from datetime import datetime
import os
import sys
dirname = os.path.dirname('__file__')

sys.path.append(os.path.join(dirname, "pkg"))
sys.path.append(os.path.join(dirname, "pkg", "preproc"))
from pkg.preproc import base

#### Objectives

1. Identify number of potential speakers (clustering)
    1.1 Use Factors: Readability (F-K), doc_length, n_mentions, n_hashtags, time_of_day??, avg_word_size, avg_num_syllables
2. Attempt to identify Trump (with 'known' data)

In [2]:
from pkg.mod import cluster
import numpy as np
from collections import Counter
from pandas import DataFrame
fields = ["n_hashtags", "n_mentions", "avg_syllables",
          "avg_word_length", "fk", 'n_sents',
          "n_ents", "n_uppers", "platform_id",'amplifier',
          'analneg', 'attribadj', 'auxdo', 'bemv',
          'bracket', 'caps', 'cconj', 'cntrstconj',
          'colon', 'comma', 'defart', 'detquan',
          'exclam', 'fstpp', 'fulstop', 'gerund',
          'havemv', 'imperative', 'indefart',
          'infinitive', 'it', 'mdnec',
          'mdposs', 'mdpred', 'multiwvb',
          'nomin', 'numdet', 'numnoun', 'objpro',
          'otheradv', 'othrintj', 'othrnoun',
          'othrverb', 'passive', 'past', 'perceptvb',
          'perfect', 'posesprpn', 'possdet', 'predadj',
          'prep', 'procontract', 'progressive',
          'proquan', 'provdo', 'prpn', 'prvv', 'pubv',
          'ques', 'relclausesubgap', 'sinflect',
          'sndpp', 'stancevb', 'subjpro', 'superlative',
          'thrdpp', 'timeadv', 'whw', 'initialmention']
dat = base.Data()
prep = base.Preprocessor()
dat.get_data(load_local=True)

prep.tokenize(data=dat.data)

ids_ = [d.ID for d in prep.docs]
arrs = [d.feature_array for d in prep.docs]
df = DataFrame(arrs, index=ids_, columns=fields)


100%|██████████| 17433/17433 [07:56<00:00, 36.62it/s]


Length Error: 
4746) Iran humiliated the United States with the capture of our 10 sailors. Horrible pictures and images. We are weak. I will NOT forget!
Length Error: 
4748) Iran humiliated the United States with the capture of our 10 sailors. Horrible pictures and images. We are weak. I will NOT forget!


In [3]:
mod0 = cluster(data=arrs, cluster_meth='optics', xi=.000075,
               metric='l2', min_cluster_size=500)

mod0_labs = mod0.labels_
print(np.unique(mod0_labs))
print(dict(Counter(mod0_labs)))

print(len(mod0_labs))

df.insert(len(df.columns), "cluster", mod0_labs)

tod_map = {}
dates, times, tokens = [], [], []
#all_tokens = []
platforms = []
for d in prep.docs:
    dates.append(datetime.strptime(d.local_date, "%Y-%m-%d"))
    tm = d.local_time
    times.append(tm)
    platforms.append(d.platform)
    tokens.append(",".join(d.get_tokens_merged(lowercase=True)))
    #all_tokens.extend(d.get_tokens_merged(lowercase=True))

df.insert(0, "tokens", tokens)
df.insert(0, 'date', dates)
df.insert(0, 'time', times)
df.insert(0, 'platform', platforms)

OPTICS(metric='l2', min_cluster_size=500, xi=7.5e-05) Loaded.
[-1  0  1]
{-1: 16298, 0: 505, 1: 514}
17317


In [4]:
from pkg.utils import make_tokens_list
from tabulate import tabulate

all_tokens = make_tokens_list(df['tokens'])
android_tokens_series = df.loc[[i for i in df.index if df.loc[i]['platform']=='android']]['tokens']
android_tokens = make_tokens_list(android_tokens_series)
iphone_tokens_series = df.loc[[i for i in df.index if df.loc[i]['platform']=='iphone']]['tokens']

iphone_tokens = make_tokens_list(iphone_tokens_series)

words_counter = Counter(all_tokens)
android_counter = Counter(android_tokens)
iphone_counter = Counter(iphone_tokens)



android_table = tabulate(list(most_common_android.items()),
                         headers=("token", "count"))
iphone_table = tabulate(list(most_common_iphone.items()),
                        headers=("token", "count"))

print("\n================  ANDROID  ================\n")
print(android_table)
print("\n================  IPHONE   ===============\n")
print(iphone_table)


================  ANDROID  ================

token          count
-----------  -------
great            589
people           289
thanks           213
big              205
obama            190
president        189
usa.             187
would            184
many             170
make             164
get              156
thank            155
time             155
bad              150
country          146
never            143
good             139
much             139
really           137
one              136
like             136
going            133
america          121
back             110
tonight          108
even              99
said              95
new               94
go                94
amazing           93
enjoy             92
win               91
job               88
today             87
money             86
media             85
poll              84
total             83
wow               83
hillary           82
made              81
want              81
show              79
got      

In [ ]:
### Difference Between IPhone and Android



In [5]:
clusts = df['cluster']

outlier_filt = clusts == -1
clust_filt = clusts != -1

df_outliers = df.where(outlier_filt)
df_clusts = df.where(clust_filt)
df_outliers = df_outliers.dropna()
df_clusts = df_clusts.dropna()
grps = df_clusts.groupby(['cluster'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from time import strptime
sns.set_theme(style="darkgrid")


fieldsx = fields.copy()
fieldsx.append('cluster')
fig = sns.pairplot(df[fieldsx], hue='cluster')

plt.show()
fig.savefig('plots/lang_attrs.png')



maybe_trump_df = df.loc[[idx for idx in df.index
                         if (df.loc[idx]['platform'] == 'android'
                             or df.loc[idx]['platform'] == 'web client')
                         or df.loc[idx]['date'] <=  datetime(2017, 1, 20)
                         or (df.loc[idx]['date'] > datetime(2017, 1, 20) and
                             13 > strptime(df.loc[idx]['time'], "%H:%M:%S").tm_hour > 10)
                         ]]







/usr/lib/python3.8/site-packages/seaborn/distributions.py:305: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/usr/lib/python3.8/site-packages/seaborn/distributions.py:305: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/usr/lib/python3.8/site-packages/seaborn/distributions.py:305: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/usr/lib/python3.8/site-packages/seaborn/distributions.py:305: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/usr/lib/python3.8/site-packages/seaborn/distributions.py:305: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/usr/lib/python3.8/site-packages/seaborn/distributions.py:305: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/usr/lib/python3.8/site-packages/s

KeyboardInterrupt: 